### Part_4_Building_WebApp_using_Anvil

In [1]:
 # Mounting the drive
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
# Change the current working Directory 
import os

path = '/content/drive/My Drive/Project/NLP'
try:   
    os.chdir(path)
    print("Directory changed to Project folder")
except OSError:
    print("Can't change the Current Working Directory")  

Directory changed to Project folder


In [3]:
import keras.backend as K
def precision(y_true, y_pred): 
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision
def recall(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

Using TensorFlow backend.


In [4]:
# Loading the best fit model
from tensorflow.keras.models import load_model

dependencies = {
    'precision' : precision,
    'recall' : recall
}

model = load_model('model_emb_bilstm_sampledData.h5', custom_objects=dependencies)

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [5]:
# Viewing the summary
model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 926)]             0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 926, 300)          4416600   
_________________________________________________________________
bidirectional_1 (Bidirection (None, 926, 600)          1442400   
_________________________________________________________________
time_distributed_1 (TimeDist (None, 926, 300)          180300    
_________________________________________________________________
flatten_1 (Flatten)          (None, 277800)            0         
_________________________________________________________________
dense_5 (Dense)              (None, 300)               83340300  
_________________________________________________________________
dense_6 (Dense)              (None, 187)               5628

In [0]:
# Loading the unique labels
import numpy as np
labels=np.load('uniqueLabels.npy',allow_pickle=True)

In [7]:
print(labels)

['GRP_0' 'GRP_1' 'GRP_10' 'GRP_11' 'GRP_12' 'GRP_13' 'GRP_14' 'GRP_15'
 'GRP_16' 'GRP_17' 'GRP_18' 'GRP_19' 'GRP_2' 'GRP_20' 'GRP_21' 'GRP_22'
 'GRP_23' 'GRP_24' 'GRP_25' 'GRP_26' 'GRP_27' 'GRP_28' 'GRP_29' 'GRP_3'
 'GRP_30' 'GRP_31' 'GRP_32' 'GRP_33' 'GRP_34' 'GRP_35' 'GRP_36' 'GRP_37'
 'GRP_38' 'GRP_39' 'GRP_4' 'GRP_40' 'GRP_41' 'GRP_42' 'GRP_43' 'GRP_44'
 'GRP_45' 'GRP_46' 'GRP_47' 'GRP_48' 'GRP_49' 'GRP_5' 'GRP_50' 'GRP_51'
 'GRP_52' 'GRP_53' 'GRP_54' 'GRP_55' 'GRP_56' 'GRP_57' 'GRP_58' 'GRP_59'
 'GRP_6' 'GRP_60' 'GRP_61' 'GRP_62' 'GRP_63' 'GRP_64' 'GRP_65' 'GRP_66'
 'GRP_67' 'GRP_68' 'GRP_69' 'GRP_7' 'GRP_70' 'GRP_71' 'GRP_72' 'GRP_73'
 'GRP_8' 'GRP_9']


In [0]:
# Importing necessary libraries
from tensorflow.keras.preprocessing.sequence import pad_sequences
import pprint, pickle
import warnings, pickle, inflect, re
import os, sys, itertools, string
from wordcloud import STOPWORDS
from nltk.tokenize import regexp_tokenize
from nltk.corpus import stopwords
import spacy
import nltk
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator

In [0]:
# loading the tokenizer
max_features = 50000
with open('tokenizer.pickle', 'rb') as handle:
    tokenizer_in_use = pickle.load(handle)

In [10]:
# Downloading the required packages
nltk.download("wordnet")
nltk.download("stopwords")
import spacy.cli
spacy.cli.download("en_core_web_lg")

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_lg')


In [0]:
# Initialize spacy 'en' large model, keeping only tagger component needed for lemmatization
nlp = spacy.load('en_core_web_lg', disable=['parser', 'ner'])
# Tokenization using Regexp_tokenizer so that we get only words and digits in the token
def tokenizer(text):
  return regexp_tokenize(text,pattern="\w+")

#Lemmatization Using Spacy
def lemmatizer(sentence):
    # Parse the sentence using the loaded 'en' model object `nlp`
    doc = nlp(sentence)
    return " ".join([token.lemma_ for token in doc if token.lemma_ !='-PRON-'])

#stop words using NLTK
def stopWordsRemoval():
  mylist = stopwords.words('english')
  # mylist = mylist.extend(['yes','na','hi','receive','hello','regards','thanks','from','greeting','will','please','see','help','able'])
  return mylist
  

  # Extend the English Stop Words for the words imported from wordnet
STOP_WORDS = STOPWORDS.union({'yes','na','hi',
                              'receive','hello',
                              'regards','thanks',
                              'from','greeting',
                              'forward','reply',
                              'will','please',
                              'see','help','able'})

# Define regex patterns
EMAIL_PATTERN = r"([\w.+-]+@[a-z\d-]+\.[a-z\d.-]+)"
PUNCT_PATTERN = r"[,|@|\|?|\\|$&*|%|\r|\n|.:|\s+|/|//|\\|/|\||-|<|>|;|(|)|=|+|#|-|\"|[-\]]|{|}]"
# Negative Lookbehind for EmailId replacement- Don't match any number which follows the text "RetainedEmailId"
NUMER_PATTERN = r"(?<!RetainedEmailId)(\d+(?:\.\d+)?)"
def cleanseText(text):
    # Make the text unicase (lower) 
    text = str(text).lower()
    # Remove email adresses
    # text = re.sub(EMAIL_PATTERN, '', text, flags=re.IGNORECASE)
    # Save Email addresses and replace them with custom keyword
    email_dict = extract_email(text)
    for key in email_dict.keys():
        text = text.replace(email_dict[key], key)
    # Remove all numbers 
    text = re.sub(NUMER_PATTERN, '', text)
    # Replace all punctuations with blank space
    # text = re.sub(PUNCT_PATTERN, " ", text, flags=re.MULTILINE)
    text = text.translate(str.maketrans("","", string.punctuation))
    text = re.sub(r'\s+', ' ', text)
    # Replace multiple spaces from prev step to single
    text = re.sub(r' {2,}', " ", text, flags=re.MULTILINE)
    text = text.replace('`',"'")
    # Replace the email ids back into their original position
    for key in email_dict.keys():
        text = text.replace(key, email_dict[key])
    return text.strip()



def extract_email(text):
    # Replaces the email addresses with custom key word and 
    # save them into a dictionary for future use
    unique_emailid = set(re.findall(EMAIL_PATTERN, text))
    email_replacement = dict()
    for idx, email in enumerate(unique_emailid):
        email_replacement[f'RetainedEmailId{idx}'] = email
    return email_replacement
  # This method callsthe cleanseTest, tokenizer and the lemmatization function

def preProcessData(text):
  cleansedText = cleanseText(text)
  tokenized = tokenizer(cleansedText)
  lemmatizedText = lemmatizer(" ".join(word for word in tokenized))
  return lemmatizedText
  
stwords = stopWordsRemoval()
STOP_WORDS.union(set(stwords))
STOP_WORDS.remove('no')
STOP_WORDS.remove('not')
# function to remove all the stopwords
def removeStopWords(text):
  return " ".join([words.strip() for words in text.split() if words.strip() not in list(STOP_WORDS)])

In [12]:
# Insalling anvil-uplink
!pip install anvil-uplink

     |████████████████████████████████| 61kB 3.3MB/s 
  Created wheel for ws4py: filename=ws4py-0.3.4-cp36-none-any.whl size=41809 sha256=c97f47edb930ff272da017833242c5b3aaa5708f1260b8ad6507cc29c93f35b7
  Stored in directory: /root/.cache/pip/wheels/19/1f/0d/beff5822af761b66067b5e0b251a9c66af3ae15828ee9a8f15
Successfully built ws4py


In [13]:
#Connecting to the server
import anvil.server

anvil.server.connect("RAJVFTEHMJWRR2MFQKYI4EAF-2CSMYUKQ544AGBV6")

Connecting to wss://anvil.works/uplink
Anvil websocket open
Authenticated OK


In [0]:
# function to predict the group

@anvil.server.callable
def predict_group(shortDescription,description):
  if not shortDescription.strip():
    shortDescription=''
  if not description.strip():
    description=''
  combinedDescription =  " ".join([shortDescription, description]).strip()
  combinedDescription = preProcessData(combinedDescription)
  combinedDescription = removeStopWords(combinedDescription)
  combinedDescription=[combinedDescription]
  print(combinedDescription)
  seq = tokenizer_in_use.texts_to_sequences(combinedDescription)
  padded = pad_sequences(seq, maxlen=926)
  from tensorflow.keras.models import load_model

  dependencies = {
      'precision' : precision,
      'recall' : recall
  }

  model = load_model('model_emb_bilstm_sampledData.h5', custom_objects=dependencies)
  pred = model.predict(padded)
  return pred,labels[np.argmax(pred)]
anvil.server.wait_forever()

The link to our webApp is :
https://nlpgroup4automaticticketclassificationsystem.anvil.app/